

Author: Jashandeep Singh 




## Data Cleansing 


We have 3 input files provided: "dirty_data.csv", "missing_data.csv". "warehouses.csv".

The task requires us to perform graphical and/or non-graphical EDA methods to understand
the data first and then find and fix the data problems.


## Initializing the Libraries required:

In [ ]:
#impoting Libraries

%matplotlib inline
from matplotlib import pyplot as plt


#system
import sys

#dataframe and array libraries 
import pandas as pd
import numpy as np

import math

#scipy 
from scipy.stats import norm

#haversine
#!pip install haversine

from haversine import haversine, Unit

#sklearn 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression 


#seaborn
import seaborn as sns

#nltk Sentiment Analyser 
from nltk.sentiment.vader import SentimentIntensityAnalyzer as salz

## Importing and loading the datasets

In [ ]:
#given that all the datasets are in the same directory as this jupyter notebook. 

dirty_data = pd.read_csv("dirty_data.csv")

missing_data = pd.read_csv("missing_data.csv")

warehouse_data = pd.read_csv("warehouses.csv")

We start by observing the "missing_data" and "warehouse_data" df to observe the column names and row values 

In [ ]:
missing_data.head(10)

In [ ]:
warehouse_data

## Missing Dataset 


### Step 1: Identifying the missing values 


In [ ]:
#total information of records. 
missing_data.info()


In [ ]:
#exploratory data analysis (EDA)

# using seaborn to visualise the missing values in missing_data
plt.figure(figsize = (16,8)) 
sns.heatmap(missing_data.isnull(),yticklabels=False,cbar=True,cmap='cividis')

It can be observed from the info data and the visualisation that the columns having the missing data in the "missing_data" dataframe are: 

"nearest_warehouse"

"order_price"

"customer_lat"

"customer_long" 

"order_total"

"season"

"distance_to_nearest_warehouse"

"is_happy_customer" 

It is also obsereved that all of these columns have 10 row inputs missing, as the total number of records is 500, and the above mentioned columns have a total count of 490.

### Step 2: Fixing the missing data for columns: 

#### A.  "is_happy_customer" 



In [ ]:
#calculating the Polaratiy score for the available data to further compare and fulfill the missing data 

# Initialising nltk SentimentIntensityAnalyser 
sentiment = salz()


Our first approach is to use the nltk sentiment analyzer to find the polarity score for the "latest_customer_review" column. Having a positive or a negative valence will help us determine the customer using a net positive or a net negative review that can in short help us find the missing "is_happy_customer" column values (True or False for Positve or Negative valence).

Identifying the outliers from the polarity scores in the column "latest_customer_review"


In [ ]:
#creating a temp dataset with indexed values of rows with null values 
happy_cust_missing = missing_data[missing_data.is_happy_customer.isnull()]

#singling out the index value list for computing the values into the missing_data df
#index values are the same as of the missing_data df
index_list = list(happy_cust_missing.index)

#initialising variable for the index list loop
#the polarity score computation for "column" goes in the same sequence of the index list values.

n = 0

if n != len(index_list):
    for column in happy_cust_missing['latest_customer_review' ]:
        
        #NLTK sentiment document reference:  https://www.nltk.org/howto/sentiment.html
        #a review is considered positive if the compound score is > 0.05
        
        if sentiment.polarity_scores(column)["compound"] > 0.05: 
                #showing the review and polarity result
                print("is_happy_customer is True for the review:","\n", column,"\n")
                
                #adding the missing "is_happy_customer" value
                missing_data.at[index_list[n], "is_happy_customer"] = True
                n = n+1


        else:
                #showing the review and polarity result
                print("is_happy_customer is False for the review:","\n", column, "\n")
                
                #adding the missing "is_happy_customer" value
                missing_data.at[index_list[n], "is_happy_customer"] = False
                n = n+1


#### B.  "nearest_warehouse" 

- Given that the radius of the earth is 6378 KM




In [ ]:
# given the function based on haversine formula to find distance between two locations 
# changed the name to hav_distance for code compatibility. 
# ref: https://edstem.org/courses/5118/discussion/375168


#changed the function name to hav_distance() for better compatibility throughout the code.



def hav_distance(origin, destination):
    lat1 , lon1 = origin
    
    lat2, lon2 = destination 
    
    radius = 6378 # radius of earth in kms
    
    
    dlat = math.radians(lat2 - lat1)
    
    dlon = math.radians(lon2 - lon1)
    
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1))* math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    
    d = radius * c
    
    return d


In [ ]:
warehouse_data

We convert the warehouse_data to a dictionary for better access of lat and long values, as having a dict with keys as the name of the warehouse and values as the combines lat and long data. 

In [ ]:
#curating positonal data lists for the warehouse locations
#df to dict ref: https://stackoverflow.com/questions/26716616/convert-a-pandas-dataframe-to-a-dictionary

warehouse_loc_dict = warehouse_data.set_index("names").T.to_dict("list")



In [ ]:
#singling out the df with missing "nearest_warehouse" values
warehouse_missing = missing_data[missing_data.nearest_warehouse.isnull()]

It is observed that the other values, such as "customer_lat", "customer_long", "distance_to_nearest_warehouse" are present for the rows where "nearest_warehouse" is Null, hence we proceed by computing and comparing the result using haversine formula (hav_distance function) to the values provided in warehouse_data.

Computing using the defined function and rounding off to 4 decimal places, the warehouse of whom, the distance is computed closest to the given "distance_to_nearest_warehouse", will be added in the missing column data.


We iterate a loop to compute a every customer lat and long, on at a time, with the 3 warehouses and curate a temporary dictionary with the calculated distances. For that particular customer the plan of approach for computing "nearest_warehouse" is to to chose the minimum distance value obtained while computing for all the warehouses. We use min() function to choose the warehouse with the minimum distance to the customer. 
 



In [ ]:
#making a list of the index values
n_warehouse_missing_index = list(warehouse_missing.index)


#Iterating a loop for curating lists of missing warehouse customer positional data 
n = 0

c_lat = []
c_long = []

if n != len(n_warehouse_missing_index):
    
    
    #all the appending is in the same sequence as the index of the rows 
    for lat in warehouse_missing["customer_lat"]:
        c_lat.append(lat)
        
        
    for long in warehouse_missing["customer_long"]:
        c_long.append(long)
        
    n = n+1

    
    
#iterating a loop for computing and adding the missing "nearest_warehouse" values.    
n = 0 

while n != len(n_warehouse_missing_index):
    distance_all = {}
    
    
    for key, values in warehouse_loc_dict.items():
        
        #rounding off the distance value to 4 decimal places
        distance_all[key] = round(hav_distance((c_lat[n], c_long[n]), warehouse_loc_dict[key]),4)
        
    #minimum value from a dict ref: https://stackoverflow.com/questions/3282823/get-the-key-corresponding-to-the-minimum-value-within-a-dictionary
    #using the min distance warehouse for filling "nearest_warehouse"
    
    missing_data.at[n_warehouse_missing_index[n], "nearest_warehouse"] = min(distance_all, key = distance_all.get)
    
    
    #printing the index and name of nearest warehouse that is computed.
    print("For Index:", n_warehouse_missing_index[n], ", the nearest warehouse is:", min(distance_all, key = distance_all.get))
    
    #resetting the dictionary 
    distance_all.clear()
    #iterating the loop ahead
    n = n+1
        
    

#### C. "distance_to_nearest_warehouse"

In [ ]:
#curating a df with null "distance_to_nearest_warehouse" values

dist_ware_missing = missing_data[missing_data.distance_to_nearest_warehouse.isnull()]

#curating an index list

dist_ware_missing_index = list(dist_ware_missing.index)

We approach to find the missing values by making a separate df with the null values and also making an index list of the made dataframe. The index list remains unique and will act as a pointer for us to make the missing data additions into the original file "missing_data"

In [ ]:
dist_ware_missing

using the hav_distance() function, we calculate the "distance_to_nearest_warehouse" value. We use warehouse_loc_dict that was created earlier, which is a dictionary with keys as name of the warehouses and values as the lat and long data.

In [ ]:
n = 0 

while n!= len(dist_ware_missing_index): 
    
    #using the "customer_lat" and "customer_long" values for rows with null "distance_to_nearest_warehouse".
    #fetching the warehouse loc values from "warehouse_loc_dict" defined in part B. 
    #computing the distance based on the "nearest_warehouse" given. 
    #rounding the distance value to 4 decimal places.
    
    distance = round(hav_distance([dist_ware_missing.loc[dist_ware_missing_index[n], "customer_lat"],  dist_ware_missing.loc[dist_ware_missing_index[n], "customer_long"]], 
                               (warehouse_loc_dict[dist_ware_missing.loc[dist_ware_missing_index[n], "nearest_warehouse"]]) ), 4) 
    
    
    print("the distance for index ",dist_ware_missing_index[n], "is: ", distance)
    missing_data.at[dist_ware_missing_index[n], "distance_to_nearest_warehouse"] = distance 
    print("correct value added")
    
    n = n+1


#### D. "customer_lat"

In [ ]:
#given function to find customer latitude
#ref: https://edstem.org/courses/5118/discussion/375168


def get_lat(dist, lat1, lon1, lon2):
    radius = 6378 #radius of earth in kms 
    
    a = -0.5 * math.sin(math.radians(lat1)) 
    b = (math.sin(math.radians(lon2-lon1)/2)** 2 - 0.5) * math.cos(math.radians(lat1))
    x_squa = math.tan(dist / radius /2) ** 2
    phi = math.atan2(b,a) ##radius 
    
    A = math.sqrt(a**2 + b**2)
    
    
    s1 = round((-phi - math.asin(((x_squa/(1-x_squa)) - 0.5)/A) - math.pi) / math.pi * 180 , 7)
    
    s2 = round((-phi + math.asin(((x_squa/ (1-x_squa)) - 0.5)/A)) / math.pi * 180 , 7)
    
    return s1, s2





using the same approach, we make a seperate df and index list 

In [ ]:
cust_lat_missing = missing_data[missing_data.customer_lat.isnull()]

cust_lat_missing_index = list(cust_lat_missing.index)

In [ ]:
cust_lat_missing

Here, we simillarly iterate a loop over the len of the index list and single out customer longitude, warehouse name, warehouse position (based on warehouse name) and distance for each row for the index in the index list. 

using the get_lat() function provided and mentioned above, we compute the customer latitude value for the specific index. We also see that the function produces 2 results and hence we observed both the results 

In [ ]:
n = 0 


while n!= len(cust_lat_missing_index):
    
    #here x = (cust_lat_missing.at[cust_lat_missing_index[n],"customer_lat"])
    
    customer_long = cust_lat_missing.at[cust_lat_missing_index[n],"customer_long"]

    ware_name = cust_lat_missing.at[cust_lat_missing_index[n],"nearest_warehouse"]

    warehouse_pos =  warehouse_loc_dict[str(ware_name)]
    
    distance = cust_lat_missing.at[cust_lat_missing_index[n], "distance_to_nearest_warehouse"]
    
    
    #using the provided get_lat function 
    
    customer_lat = get_lat(distance, warehouse_pos[0], warehouse_pos[1], customer_long)
    
    
    print("for index value:", n, "customer_lat values are:", customer_lat)

    #using the distance formula to find the best option of the latitudes. 
    
    distance_1 = hav_distance((customer_lat[0], customer_long),(warehouse_pos))
    print("for customer_lat 1 value , distance1: ", round(distance_1, 4))
                              
    distance_2 = hav_distance((customer_lat[1], customer_long),(warehouse_pos))
    print("for customer_lat 2 value distance2: ", round(distance_2, 4))
    
    #since both the latitude values generate the same rounded distance, we choose one value to update in the df
    
    # index values as the guide to update "missing_data"
    missing_data.at[cust_lat_missing_index[n], "customer_lat"] = customer_lat[0]
    
    print("for index value ", n , "customer_lat value added")
    
    
    
    n = n + 1

It is clearly obsereved that both the results are the same so we chose a single latitude point and added in the "missing_data"

#### E. "customer_long"

In [ ]:
# given the  fucntion to find customer longitude 
# ref: https://edstem.org/courses/5118/discussion/375168



def get_lon(dist, lat1, lon1, lat2):
    
    radius = 6378 #radius of earth in kms 
    
    x_squa = math.tan(dist / radius/2 )** 2
    top = math.sqrt((x_squa / (1+ x_squa) - math.sin(math .radians(lat2 - lat1)/2) ** 2)**2)
    bot = math.cos(math.radians(lat1)) * math.cos(math.radians(lat2))
    
    s1 = round(2 * math.asin(math.sqrt(top/bot)) / math.pi * 180 + lon1, 7)
    
    s2 = round(2 * math.asin(-math.sqrt(top/bot)) / math.pi * 180 + lon1, 7)
    
    return s1, s2

using the same approach, curting different df and index list

In [ ]:
cust_long_missing = missing_data[missing_data.customer_long.isnull()]

cust_long_missing_index = list(cust_long_missing.index)

In [ ]:
cust_long_missing

Here, similar to part D, we find the customer longitude values by using the provided get_long() funtion. Using the same ideology and approach as part D, we single out all the values (here customer latitude values) and calculate the longitude values. Also, as same, the function produces 2 values, for which, both the distances were same, and hence one values was added in the "missing_data" 

In [ ]:
n = 0 


while n!= len(cust_long_missing_index):
    
    
    
    customer_lat = cust_long_missing.at[cust_long_missing_index[n],"customer_lat"]

    ware_name = cust_long_missing.at[cust_long_missing_index[n],"nearest_warehouse"]

    warehouse_pos =  warehouse_loc_dict[str(ware_name)]
    
    distance = cust_long_missing.at[cust_long_missing_index[n], "distance_to_nearest_warehouse"]
    
    
    #using the provided get_long function 
    
    customer_long = get_lon(distance, warehouse_pos[0], warehouse_pos[1], customer_lat)
    
    
    print("for index value:", n, "customer_long values are:", customer_long)

    #using the distance formula to find the best option of the latitudes. 
    
    distance_1 = hav_distance((customer_lat, customer_long[0]),(warehouse_pos))
    print("for customer_lat 1 value , distance1: ", round(distance_1, 4))
                              
    distance_2 = hav_distance((customer_lat, customer_long[0]),(warehouse_pos))
    print("for customer_lat 2 value distance2: ", round(distance_2, 4))
    
    #since both the longitude values generate the same rounded distance, we choose one value to update in the df
    
    
    # index values as the guide to update "missing_data"
    missing_data.at[cust_long_missing_index[n], "customer_long"] = customer_long[0]
    
    print("for index value ", n , "customer_lat value added")
    
    
    
    n = n + 1

#### F.  "season" 

using the same approach, to make a different df and index list 

In [ ]:
# making a df with season null values 
season_missing = missing_data[missing_data.season.isnull()]

#index list for missing season value rows
season_missing_index = list(season_missing.index)

In [ ]:
season_missing

In [ ]:
missing_data["season"].unique()

It is observed that our dataset contains 4 seasons "Autumn", "Summer", "Spring", "Winter". Accoring to the classification given, 


    In Australia, the seasons are defined by grouping the calendar months in the following way:

    Spring - the three transition months September, October and November.
    Summer - the three hottest months December, January and February.
    Autumn - the transition months March, April and May.
    Winter - the three coldest months June, July and August.


We proceed by adding a column named "month" in "season_missing" df, that singles out the month for the date provided in the same row. Our plan of action is to match the month for that row for an integer range set to define the season. 

In [ ]:
season_missing["month"] = pd.DatetimeIndex(season_missing['date']).month

#ignore the warning, raised for creating a copy on the slice of original df

We define a function "season_finder()" which returns the name of the season based on the month value in the date. The range for the month values for deciding the season is based upon the information given in the assignment instructions. 

In [ ]:
#defining a function that returns season name with month number as input 

def season_finder(month_number):
    
    #for months March, April and May
    if month_number in range(3,5):
        return "Autumn"
    
    #for months June, July and August
    if month_number in range(6,8):
        return "Winter"
    
    #for months September, October and November
    if month_number in range(9,11):
        return "Spring"
    
    #for months December, January and February
    else:
        return "Summer"
    


iterating a loop limiting over length of index list, for each missing season value in the row, we compute season from the added "month" column and add the missing season value in the "missing_data" for the same index. 

Also, since the "month" column is added in our speratly created df, our "missing_data" df is not affected.

In [ ]:

#initialising variable for while loop 
n = 0 

while n!= len(season_missing_index):
    
    #using "season_finder" funtion fetching the season name for the missing "season" value row.
    season_name = season_finder(season_missing.at[season_missing_index[n], "month"])
    
    
    
    print("the season for index ",season_missing_index[n], "is: ", season_name)
    
    #adding the missing "season" value into the "missing_data" df using the same index value. 
    missing_data.at[season_missing_index[n], "season"] = season_name
    
    print("value addded", "\n")
    
    n = n + 1 

#### G. "order_total"

using the same approach, making a different df and index list

In [ ]:
#making a df with order_total null values
order_total_missing = missing_data[missing_data.order_total.isnull()]

#index list for missing "order_total" value rows
order_total_missing_index = list(order_total_missing.index)

In [ ]:
order_total_missing

We observed that order_total is calculated by using the formula (coupon_discount is given in %): 

    order_total = order_price - (order_price * (coupon_discount/100)) + delivery_charges
    
Hence, we proceed by calculating the "order_total" for the rows with missing "order_total" values and also rounding them to 2 decimal values


iterating over a loop limiting on length of index list, we fetch all teh variables required by teh formula and then compute the order total. Also, rounding the result value to 2 decimal points, maintaing unity in our "missing_data" df


In [ ]:
#initiialising variable for while loop 

n =0 

while n != len(order_total_missing_index): 
    
    #order_price value for the specific index value row
    order_price = order_total_missing.at[order_total_missing_index[n], "order_price"]
    
    #coupon_discount value 
    coupon_discount = order_total_missing.at[order_total_missing_index[n], "coupon_discount"]
    
    #delivery_charges value 
    delivery_charges =  order_total_missing.at[order_total_missing_index[n], "delivery_charges"]
    
    
    #calculating order_total value 
    order_total = round((order_price - (order_price * (coupon_discount/100)) + delivery_charges), 2)
    
    #print value check
    print("the order_total for index ",order_total_missing_index[n], "is: ", order_total)
    
    #adding the missing "order_total" value to "missing_data" df
    missing_data.at[order_total_missing_index[n], "order_total"] = order_total
    
    #printing confirmation
    print("value added", "\n")
    
    
    n = n + 1 

#### H. "order_price"

using same approach, making different df and index list 

In [ ]:
#df with order_price null values 
order_price_missing = missing_data[missing_data.order_price.isnull()]

#index list for missing "order_price" value rows
order_price_missing_index = list(order_price_missing.index)

In [ ]:
order_price_missing

Similar to part G., re-arranging the observed formula, we calculate "order_price" by the following formula (coupon_discount is given in %): 

    order_price = (order_total - delivery_charges)/(1 - (coupon_discount/100))
    
Hence, we proceed to calculate the missing "order_price" values by iterating a loop limiting on the length of the index list and fetching all the variables required by the formula and computing the result. Also, rounding the result to 1 decimal place to maintain unity over the df. (Although all the values are integers)

In [ ]:
#initialising the variable for the loop
n =0 

while n != len(order_price_missing_index): 
    
    #order_total value for the specific index value row
    order_total = order_price_missing.at[order_price_missing_index[n], "order_total"]
    
    #coupon_discount value 
    coupon_discount = order_price_missing.at[order_price_missing_index[n], "coupon_discount"]
    
    #delivery_charges value 
    delivery_charges =  order_price_missing.at[order_price_missing_index[n], "delivery_charges"]
    
    
    #calculating order_price value 
    order_price = round((order_total - delivery_charges)/(1 - (coupon_discount/100)), 1)
    
    #print value check
    print("the order_price for index ",order_total_missing_index[n], "is: ", order_price)
    
    #adding the missing "order_price" value to "missing_data" df
    missing_data.at[order_price_missing_index[n], "order_price"] = order_price
    
    #printing confirmation
    print("value added", "\n")
    
    
    n = n + 1 

### Step 3: Output the corrected file

- saving as 'missing_data_solution.csv'



using the info() function, we check any irregularities in our analysis. It is observed that all the null values are added, the total non-null count is 500. The number of columns are same as the original file. We also cross check if any syntactic errors are made in the df by comparing the values over unique() function. After no errors or missing data is observed, we create the output file.  

In [ ]:
missing_data.info()

In [ ]:
missing_data.nearest_warehouse.unique()

In [ ]:
missing_data.season.unique()

In [ ]:
 # Saving the final df with all the missing data corrections
missing_data.to_csv('missing_data_solution.csv',header=True)

## Dirty Dataset 

Similar to Missing Data Dataset, we start by observing the columns and row values of the dirty_data

In [ ]:
dirty_data.info()

In [ ]:
dirty_data


#### Our Plan of Action will be to modulate different functions that handle the errors in the data and then running all those functions over a loop interating over the index values in the dirty_data file . The function we define will be checking  the possible syntactic, semantic, and coverage errors and will fix them. 




#### 1. Finding the price for every product in the shopping_cart

We start by obtaining a direct knowledge of the price of products in the "shopping_cart" column. The approach to obtain is to make a separte value matrix of the products in the shopping_cart (having products define as the column headers) and the order_price. 

In [ ]:
from itertools import chain

set(chain(missing_data.shopping_cart))
# obsering the initial element of every value in the set. 

After observing the shopping cart, it was concluded that there are only the following 10 products in the shopping cart: 

"Lucent 330S",

"iAssist Line",

"pearTV",

"Alcon 10",

"Universe Note",

"Olivia x460",

"iStream",

"Toshika 750",

"Candle Inferno",

"Thunder line"




We choose 200 smallest order_price number (choosen in order), the reason being to increase the efficiency of our value_matrix model to make equations that are compared with the products and the ammount of order of products (the number of a product acts as a linear definer for the equation to be then compared to the value_matrix of the order_price)

By using the numpy.linalg least square, we can compute linear equations made by these matrices and the computation will bring us the unique price for each product. 


In [ ]:
#Finding the price of every product in the shopping cart using the filled "missing_data" 
# We use "missing_data" as we can rely on the values more than the "dirty_data"




#creating the shopping products df 
column_names = ["Lucent 330S","iAssist Line","pearTV","Alcon 10","Universe Note","Olivia x460","iStream","Toshika 750","Candle Inferno","Thunder line"]

#keeing the range 200 for each product in the matrix 
df = pd.DataFrame(0,index=range(0,200),columns = column_names)

#creating an order price value matrix
#defining 200 size for better equations computation

order_array = missing_data.nsmallest(200,'order_price')
m = order_array

price = dict()
count = 0

for index,row in order_array['order_price'].iteritems():
    
    row1 =  missing_data.loc[index,'shopping_cart']
    
    for i in eval(row1): 
        df.loc[count,i[0]] = i[1]
    count = count + 1

    
#creating a value matrix for each product
df_product_array = df[["Lucent 330S","iAssist Line","pearTV","Alcon 10","Universe Note","Olivia x460","iStream","Toshika 750","Candle Inferno","Thunder line"]].to_numpy()
order_array = order_array['order_price'].to_numpy().reshape(-1, 1)


# computing value for each product 

round(np.linalg.lstsq(df_product_array,order_array, rcond= None)[0][1].tolist()[0])

# for k,i in np.linalg.lstsq(df_array,order_array)[0]:
for idx,item in enumerate(column_names):
    
    #price dict having the price for the products in the shopping cart
    price[item]=round(np.linalg.lstsq(df_product_array,order_array, rcond=None)[0][idx].tolist()[0])
    
    


In [ ]:
#viewing the price of products
price

#### 2. "date" and "season" value check function

the "date_season_check()" function takes date, season and index arguments.

We proceed to check the date and season values. The plan of action is as follows: 

1. Using pd.to_dataframe() function, we transform all date values to the '%Y-%m-%d' format. The date values that doesn't transform are then handled in the funtion. 

2. Performing a syntactic check for the "season" column values. For the irregularities such as upper or lowercase entries, we unifrom them using the capitalize() function.

3. By creating a season_dict, we'll compare the month values from the "date" to see the season name and then perform a check over the dirty_data and correct the error. 

4. By handling the ValueError() raised by irregular date format, we change the date format to '%Y-%m-%d' and update in the dirty_data file. 


In [ ]:
# correcting date format 

import datetime


#converting using to_datetime such that all the exceptions are handled by the below defined function 
dirty_data['date'] = pd.to_datetime(dirty_data['date']).dt.strftime('%Y-%m-%d')


#converting the season values to same format 
dirty_data['season'] = [x.capitalize() for x in list(dirty_data['season'])]

def date_season_check(date,season,index):
    
    #using the information given in the requirements, defining a dict with season and specific months 
    season_dict={'Summer':[12,1,2],'Autumn':[3,4,5],'Winter':[6,7,8],'Spring':[9,10,11]}
    
    
    # given date format 
    date_format = "%Y-%m-%d"
    

    season_list = season_dict.get(season)
    
    try:
        date_obj = datetime.datetime.strptime(date, date_format) 
        date_month = date_obj.month
    
        # checking and correcting for wrong season 
        if season_list:
            if date_month not in season_list:
                correct_season = [a for a, b in season_dict.items() if date_month in b]

                dirty_data.loc[index,'season'] = correct_season[0]
                
                #print to view changes
                print("season updated for index: ", index, "is ", correct_season[0])
                return True
    
    except ValueError:
        
        # correcting the format of the date using exception handling 
        correct_date=""
        
        # if date has year in starting, then day and month are swaped to form correct date
        
        if '2019' in date.split('-')[0]:
            correct_date = date.split('-')[0]
            correct_date = correct_date + '-' + date.split('-')[2] 
            correct_date = correct_date + '-' + date.split('-')[1]
            
        elif '2019' in date.split('/')[0]:
            correct_date = date.split('/')[0]
            correct_date = correct_date + '-' + date.split('/')[2] 
            correct_date = correct_date + '-' + date.split('/')[1]


        #for format with year value at last 
        elif '2019' in date.split("-")[2]: 
            correct_date = date.split('-')[2]
            
            # choose month according to given season from day and month.
            
            #for the format with month in the middle 
            if date.split('-')[1] in season_list:

                correct_date = correct_date + '-' + date.split('-')[0]
                correct_date = correct_date + '-' + date.split('-')[1] 
                
                
            #for the format with day in the middle
            else:
                correct_date = correct_date + '-' + date.split('-')[1]
                correct_date = correct_date + '-' + date.split('-')[0]

        date_obj1 = datetime.datetime.strftime(datetime.datetime.strptime(correct_date, date_format),date_format)
        
        #updating in the dirty_data
        dirty_data.at[index,'date'] = date_obj1
        
        #print to view changes 
        print("date updated for index: ", index, " ", date_obj1)
        
        return True
    return False

#### 3. "customer_lat", "customer_long", "nearest_warehouse", "distance_to_nearest_warehouse" check funtion

the "warehouse_dict_check()" takes 5 arguments which are the latitude, longitude, distance_to_nearest_warehouse, nearest_warehouse and index value. We proceed to check and correct the errors as follows:

1. Correcting the format of "nearest_warehouse" values to the original format using the capitalize() function. 

2. perfomring a value check to see wether the lat or long values are not 0.

3. performing a check to see wether the "nearest_warehouse" values are the only ones given in "warehouse_data" file. (only 3 names). Also seeing wether the name is not something else. 

4. performing a check to see the distance values matching to requirement and not 0. if not equal to computed distance, correction is made.

5. updating the corrections made in the dirty_data file and also rounding off the required values. 

In [ ]:
#changing the format of the names as required. 
dirty_data['nearest_warehouse'] = [x.capitalize() for x in list(dirty_data['nearest_warehouse'])]


def warehouse_dist_check(lat,long,warehouse_dist,nearest_warehouse,index): 
    dist = 0.0
    columns = list(warehouse_data)
    
    # swap the lat and long, by checking their values
    
    #making sure lat and long are not zero 
    if (lat>0 or long<0): 
        
        dirty_data.loc[index,'customer_lat'] = long
        dirty_data.loc[index,'customer_long'] = lat 
        return True
    
    else:
        
        cust_lat_long = [lat,long]
        
        #checking if warehouse name not in the provided warehouse_data 
        if (nearest_warehouse[0] not in 'NTB'):
            dirty_data.at[index,'nearest_warehouse'] = nearest_warehouse
            return True
        
        
        #warehouse_lat_long =  warehouse_loc_dict[str(nearest_warehouse)]
        ware_lat_long = (warehouse_data[warehouse_data['names']==nearest_warehouse]['lat'].astype(float),warehouse_data[warehouse_data['names']==nearest_warehouse]['lon'].astype(float))

        #print ("warehouse locations are:",warehouse_lat_long, "\n")
        #print('customer locations are',cust_lat_long)
        
        
        
        for index1,row in warehouse_data.iterrows():
            ware_lat_long = list() 
            ware_lat_long.append(warehouse_data[columns[1]][index1])
            ware_lat_long.append(warehouse_data[columns[2]][index1]) 
            
            
            if (dist == 0):
                
                dist = haversine((ware_lat_long),(cust_lat_long))
                warehouse = warehouse_data.loc[index1,'names']
                
            elif (dist > haversine(ware_lat_long,cust_lat_long)):
                
                dist = haversine(ware_lat_long,cust_lat_long)
                
                warehouse = warehouse_data.loc[index1,'names']
                
        if (format(warehouse_dist,'.1f') != format(dist,'.1f')): 
            dirty_data.loc[index,'distance_to_nearest_warehouse'] = round(dist,4) #rounding to 4 decimal places  
            
            
            #print to view changes 
            print("distance_to_nearest_warehouse updated for index: ", index, "is ", round(dist,4))
            return True
            
    return False


#### 4. "latest_customer_review" and "is_happy_customer" check function


the "happy_customer_check()" function takes "customer_review", "is_happy_customer" and index as arguments.

the funtion performs the following check and correction:

1. seeing over the index value, if the latest_customer_review is actually positive or negative using the nltk sentiment analyser with compound threshold of 0.05. If the review is positive but the "is_happy_customer" is False, the correction is made and same logic in the opposite case of review being negative and "is_happy_customer" True

In [ ]:
def happy_customer_check(customer_review,is_happy_customer,index):
    
    #using the NLTK sentimentanalyser initialised as 'sentiment'   
    #compung threshold of 0.05. above values are positve and below are negative polarity. 

    if sentiment.polarity_scores(customer_review)["compound"] > 0.05:
        
        #checking the value 

        if (is_happy_customer != True): 
            
            #error correction
            dirty_data.loc[index,'is_happy_customer'] = True
            
            #printing to view changes 
            print("is_happy_customer = True  updated for index: ", index)
    else:
        if (is_happy_customer != False):
            dirty_data.loc[index,'is_happy_customer'] = False
            
            #printing to view changes 
            print("is_happy_customer = False  updated for index: ", index)
            

#### 5. "order_price", "delivery_charges", "coupon_discount", "order_total" check function 

the "order_price_check()" function performs the following checks and correct the error: 

1. calculating the "order_price" value based over the "price" dict (which contains all the individual pricing for each product). 

2. correcting the "order_price" value is the computation results in different value. 

3. calulating and correcting the "order_total" for where errors were present

In [ ]:
def order_price_check(price_item,delivery_charges,coupon_discount,purch_order,index):
    
    
    row1 = dirty_data.loc[index,'shopping_cart'] 
    #print(row1)
    order_price_cal = 0
    
    
    #lower casing the elemets and the names from the price dict for perfect match 
    product_price_lowercase = {k.lower(): v for k, v in price.items()}
        
    
    for i in eval(row1):
    
        #using the product pricing 
        #lowercasing the irregularities in the product name in the "shopping_cart"
    
        order_price_cal = order_price_cal + int(product_price_lowercase.get(i[0].lower()))*int(i[1])

    if (price_item != order_price_cal):
        order_total_new = order_price_cal - (coupon_discount)*(order_price_cal/100) + delivery_charges
        dirty_data.loc[index,'order_total'] = order_total_new 
        
        #printing to view change 
        print("order_total updated for index: ", index, "is ",order_total_new)
        return True

    else:
        #checking for correct calculation of order_price with other depending elements such as cupoun code and delivery charge 
        
        order_price = purch_order + (coupon_discount)*(price_item/100) - delivery_charges
        dirty_data.loc[index,'order_price'] = round(order_price)
        
        #printing to view changes 
        print("order_price updated for index: ", index, "is ", round(order_price))
        
    return False

#### 6. Function to nest all the error correction functions. 

the "clean_data()" function takes the index as the argument and iterates for every column. When the conditions set over column names is satisfied, the function nests the above defined function and performs the analysis.

In [ ]:
#difining the function to nest all the above defined functions for correcting the dirty_data 

#using index_1 naming to erradicate any mismatch in nested function computation

def clean_data(index_1):
    columns = list(dirty_data)
    for column in columns:
        
        #checking for date or season
        if "date" in column or "season" in column:
            if (date_season_check(dirty_data.loc[index_1, "date"], dirty_data.loc[index_1, "season"], index_1)):
                continue
                
        #checking for customer lat, customer long, distance and nearest_warehouse
        if "customer_lat" in column or "distance_to_nearest_warehouse" in column:
            if (warehouse_dist_check(dirty_data.loc[index_1, "customer_lat"], dirty_data.loc[index_1,"customer_long"], dirty_data.loc[index_1,"distance_to_nearest_warehouse"], dirty_data.loc[index_1,"nearest_warehouse"], index_1)):
                continue
        
        #checking for order price or delivery charges
        if "order_price" in column or "delivery_charges" in column:
            if (order_price_check(dirty_data.loc[index_1, "order_price"], dirty_data.loc[index_1, "delivery_charges"], dirty_data.loc[index_1, "coupon_discount"], dirty_data.loc[index_1, "order_total"], index_1)):
                continue
                
        if "is_happy_customer" in column:
            happy_customer_check(dirty_data.loc[index_1, "latest_customer_review"], dirty_data.loc[index_1, "is_happy_customer"] ,index_1)

### Function Call for "clean_data( )" function to start the cleaning of the "dirty_data"

- also producing a change log made in the dirty_data with the type, value, index value. 

In [ ]:
#function call to clean the dirty data 

print("The Change Log for the dirty_data file is follows: ", "\n")

for index, row in dirty_data.iterrows():
    clean_data(index)

### Output of the corrected "dirty_data"

- performing a safety check using info() and unique() function to observe the irregularities are cleaned and no new anomolies are added.

- ouput the cleaned file with the required naming 'dirty_data_solution.csv'

In [ ]:
dirty_data.info()

In [ ]:
dirty_data.date.unique()

In [ ]:
dirty_data.nearest_warehouse.unique()

In [ ]:
dirty_data.season.unique()

In [ ]:
#output cleaned file 

dirty_data.to_csv('dirty_data_solution.csv',header=True)

In [1]:
pip install jupyterthemes


     |████████████████████████████████| 7.0 MB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 6.6 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [8]:
!jt -t monokai